## Working with an LLM programmatically

You have certainly interacted before with a Large Language Model (LLM) like ChatGPT. This is usually done through a UI or an application.

In this Notebook, we are going to use Python to connect and query an LLM directly through its API. For this Lab we have selected the model **Granite-3.1-8B-Instruct**.(https://huggingface.co/RedHatAI/granite-3.1-8b-instruct). This is a fully Open Source model (Apache 2.0 license) developed by IBM Research.

This model has already been deployed on the Lab cluster because even if it's a smaller model, it still needs a GPU with 24GB of RAM to run...

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages. We will then import the libraries we need.

In [1]:
# Step 1: Install necessary libraries (run in a cell if needed)
!uv pip install -r requirements.txt

Using Python 3.11.11 environment at: /opt/app-root
Audited 3 packages in 5ms


In [2]:
# Uncomment the following line only if you have not selected the right workbench image, or are using this notebook outside of the workshop environment.
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt
import json

# HTTP client for API calls
import httpx

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Langchain

Langchain (https://www.langchain.com/) is a framework for developing applications powered by language models. It will take care for us of all the boilerplate code we would have to manually write to properly query an LLM API.

We will start by creating an **llm** instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model. For example, `max_new_tokens` will instruct the model to answer with a maximum of 512 tokens (words or parts of words). `temperature`, set really low here, will instruct the model to stay truth-grounded, and not try to be too "creative". After all, we're not trying to write a fancy poem here!

In [3]:
# LLM Inference Server URL
inference_server_url = "http://llama3-2-3b-predictor.llama-serving.svc.cluster.local:8080/v1"
inference_server_model_name = "llama3-2-3b"


# LLM definition
llm = ChatOpenAI(
    model=inference_server_model_name,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key="EMPTY",  # if you prefer to pass api key in directly instaed of using env vars
    base_url=inference_server_url,
    http_client=httpx.Client(verify=False)    # Because we are using an internal API endpoint (service) we need to disable SSL certificate checking.
)

We also need a **template** to be applied to every request we are sending to the model (the "Prompt").

When querying a model, you almost never want to send directly what the user has typed. On top of this entry, you need to give proper instructions to the model so that it knows how to handle it: what and how to answer, what NOT to answer, the tone it must use...

In [4]:
# Prompt template
template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """You are a helpful, respectful, and honest assistant.
        Answer each question clearly and concisely in a single response only.
        Do not continue the conversation or simulate dialogue unless explicitly asked.
        Never include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
        Ensure that your responses are socially unbiased and positive in nature.
        If a question does not make sense or is not factually coherent, explain why instead of trying to answer.
        If you don't know the answer to a question, say "I don't know"."""
    ),
    HumanMessagePromptTemplate.from_template("{input}"),
])

query = "What is Artificial Intelligence?"

# ✅ Generate the prompt first
prompt = template.invoke({"input": query})

We are now ready to query the model!

In [5]:
# Convert ChatPromptValue -> list of messages
response = llm.invoke(prompt.to_messages())
print(response.content)

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. Learning: AI systems can learn from data and improve their performance over time.
2. Problem-solving: AI systems can analyze problems and find solutions.
3. Reasoning: AI systems can draw conclusions based on data and rules.
4. Perception: AI systems can interpret and understand data from sensors and other sources.

AI systems can be categorized into two main types:

1. Narrow or Weak AI: Designed to perform a specific task, such as image recognition, speech recognition, or playing chess.
2. General or Strong AI: A hypothetical AI system that possesses human-like intelligence and can perform any intellectual task.

AI is used in various applications, including virtual assistants, self-driving cars, healthcare, finance, and more.


Some information, like the tokens that were consumed and generated are present in the metadata. That can be useful to measure the consumption of the model.

In [6]:
print(json.dumps(response.usage_metadata, indent=2))

{
  "input_tokens": 154,
  "output_tokens": 172,
  "total_tokens": 326,
  "input_token_details": {},
  "output_token_details": {}
}


---
# End of activity
Return to the Roadshow instructions for next steps.